<a href="https://colab.research.google.com/github/AlexKressner/Business_Intelligence/blob/main/openai_api_chat_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# installation
! pip install -q --upgrade cohere typing-extensions==4.5.0 openai tiktoken python-dotenv beautifulsoup4

In [ ]:
from openai import OpenAI
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os

In [ ]:
# Lädt die Umgebungsvariablen aus der .env-Datei
load_dotenv()

# Zugriff auf die Umgebungsvariablen
secret_key = os.getenv('OPENAI_API_KEY')

In [ ]:
client = OpenAI(api_key=secret_key)

In [ ]:
def get_website_text(url: str):
    response = requests.get(url)

    # Überprüfen, ob die Anfrage erfolgreich war
    if response.status_code == 200:
        # Parsen des HTML-Inhalts der Seite
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extrahieren sämtlichen Textes der Webseite
        text = soup.get_text(separator=' ', strip=True)
        return text
    else:
        # Zurückgeben einer Fehlermeldung mit dem HTTP-Statuscode
        return f"Fehler beim Abrufen der Webseite: HTTP-Statuscode {response.status_code}"

In [ ]:
def get_website_urls(url: str):
    response = requests.get(url)

    # Überprüfen, ob die Anfrage erfolgreich war
    if response.status_code == 200:
        # Parsen des HTML-Inhalts der Seite
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extrahieren sämtlichen Textes der Webseite
        hyperlinks = soup.find_all('a')
        return [link.get('href') for link in hyperlinks if link.get('href')]
    else:
        # Zurückgeben einer Fehlermeldung mit dem HTTP-Statuscode
        return f"Fehler beim Abrufen der Webseite: HTTP-Statuscode {response.status_code}"

In [ ]:
def get_link_by_substring(strings: [str], substrings: [str]):
    # Konvertierung der Substrings in Kleinbuchstaben für case-insensitive Suche
    substrings_lower = [substring.lower() for substring in substrings]

    # Filtern der Liste durch Überprüfung, ob irgendein Substring in jedem Element enthalten ist
    filtered_list = [s for s in strings if any(substring_lower in s.lower() for substring_lower in substrings_lower)]

    return filtered_list

In [ ]:
# Hilfsfunktion zur Interaktion mit der Chat-API
def get_completion(messages: [str], model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content

In [ ]:
applicant_profile = """
Name des Bewerbers ist Max Mustermann; abgeschlossenes Abitur mit Note 1,7; Leistungskurse Mathematik und Physik;
Interesse an digitalen Geschäftsmodellen im Handel; Erstes Praktikum in einem Startup, das über einen online-shop schmuck verkauft;
Programmiererfahrung in Python, insbesondere Webprogrammierung; motiviert und zuverlässig
"""

In [ ]:
system_message = f"""
Du bist ein Experte bei der Erstellung professioneller Anschreiben für Bewerbungen! Du erstellst ein Anschreiben für das in einfachen Anführungszeichen beschriebene
Bewerberprofil.

'{applicant_profile}'
"""

In [ ]:
def add_job_description_to_prompt(job_description: str):
    prompt = f"""
    Du erhälst in einfachen Anführungszeichen einen Ausschreibungstext für ein duales Studium. Bitte verwende das Bewerberprofil, um ein Anschreiben für eine Bewerbung
    auf die Stellenausschreibung zu erstellen. Das Anschreiben soll professionell geschrieben sein. Das Anschreiben soll spezifisch auf die Anforderungen in der
    Ausschreibung eingehen und zeigen, dass der Bewerber diese bestmöglich erfüllt. Die Länge des Anschreibens soll eine DIN-A4 Seite sein.

    '{job_description}'
    """
    return prompt

In [ ]:
links = get_website_urls("https://www.lappcareer.com/stellenangebote.html?")

In [ ]:
relevant_links = get_link_by_substring(links,["duales studium","duales-studium", "dual", "dhbw"])
print(f"{len(relevant_links)} passende Stellenausschreibungen gefunden!")

In [ ]:
job_descriptions = []
for link in relevant_links:
    job_descriptions.append(get_website_text(link))

In [ ]:
cover_letter = []

In [ ]:
for job in job_descriptions:
    messages =  [
    {'role':'system',
    'content': system_message},
    {'role':'user',
    'content': add_job_description_to_prompt(job)},
    ]
    cover_letter.append(get_completion(messages))

In [ ]:
print(cover_letter[0])